In [1]:
try:
  import re
  import json
  from IPython.core.display import HTML
  import pandas as pd
  from base64 import b64decode
  from bs4 import BeautifulSoup as bs
except ModuleNotFoundError as e:
  %pip install pandas
  %pip install beautifulsoup4
  %pip install markdown2

In [2]:
fn = 'neural_nets.ipynb'
with open(fn) as f:
  js = json.load(f)['cells']
  df = pd.DataFrame(js)

In [3]:
def writePNG(txt, filename):
  try:
    with open(filename, 'wb') as f:
      f.write(b64decode(txt))
  except Exception as e:
    return False
  return True

In [4]:
def getHTML(row,
            texCapsul_s='<p class="katex">\n$$\n',
            texCapsul_e="\n$$\n</p>\n",
            codeCapsule_s="\n<div>\n<pre>\n<code class='language-python'>",
            codeCapsule_e="\n</code>\n</pre>\n</div>\n"):

    htm = bs('\n<div>\n</div>', 'html.parser')
    if row['cell_type'] == 'code':
        text = ''
        text += codeCapsule_s
        for s in row['source']:
            if s.strip() == '':
                continue

            text += s
        text += codeCapsule_e
        htm.append(bs(text))
    if row['cell_type'] == 'markdown':
        headerDict = {'#'*i: [f'\n<h{i}>', f'</h{i}>\n'] for i in range(1, 7)}
        found = False
        eq = [texCapsul_s]
        for s in row['source']:
            if s.startswith('\\begin{equation}') or \
                    s.startswith('\\begin{equation*}'):
                found = True
                eq.append(s)
            elif found and (s.startswith('\\end{equation}') or
                            s.startswith('\\end{equation*}')):
                eq.append(s)
                eq.append(texCapsul_e)
                htm.append(bs(''.join(eq)))
                found = False
                eq = [texCapsul_s]
            elif found:
                eq.append(s)
            else:
                x = ''
                if s.strip() == '':
                    continue
                if s.strip()[0] == '#':
                    k = s.count('#')*'#'
                    x = headerDict[k][0] + s.replace(k, '') + headerDict[k][1]
                else:
                    x = f'\n<span>{s}</span>\n'
                htm.append(bs(x))
    return htm


def ipynb2html(inFile,
               outFile='',
               tempDoc='doc_temp.html',
               mrkdnTmpl='{"cells": [{"cell_type": "markdown", "metadata": {}, "source": []}], "metadata": {"language_info": {"name": "python"}, "orig_nbformat": 4}, "nbformat": 4, "nbformat_minor": 2}',
               texCapsul_s='<p class="katex">\n$$\n',
               texCapsul_e="\n$$\n</p>\n",
               codeCapsule_s="\n<div>\n<pre>\n<code class='language-python'>",
               codeCapsule_e="\n</code>\n</pre>\n</div>\n",
               navF='../html/nav.html',
               cssDir='css',
               jsDir='js',
               htmlDir='html',
               title='title'):

    with open(inFile) as f:
        df = pd.DataFrame(json.load(f)['cells'])
        f.close()
        df['html'] = df.apply(getHTML, axis=1)
        with open(tempDoc) as f:
            page = bs(f, 'html')
        with open(navF) as f:
            nav = bs(f, 'html')
        for a in nav.find_all('a'):
            a['href'] = htmlDir + a['href'].split('/')[-1]
        for h in page.body.find_all('script'):
            if not h['src'].startswith('http'):
                h['src'] = jsDir + h['src'].split('/')[-1]
        for h in page.head.find_all('link'):
            if not h['href'].startswith('http'):
                h['href'] = cssDir + h['href'].split('/')[-1]
        page.find('div', id='navbar').append(nav)
        page.head.title.string = title
        page.find(class_='title').string = title
        for h in df.html:
            if len(h) > 1:
                page.find('div', id='dynamic_content').append(h)
        if outFile == '':
            outFile = '.'.join(inFile.split('.')[:-1]) + '.html'
        with open(outFile, "w", encoding='utf-8') as file:
            file.write(str(page))


In [5]:
navF = '../docs/View/html/nav.html'
cssDir = '../css/'
jsDir = '../js/'
htmlDir = '../html/'
fn = '../index.ipynb'
ipynb2html(fn,
           '../docs/View/html/index.html',
           jsDir=jsDir,
           htmlDir=htmlDir,
           cssDir=cssDir,
           navF=navF,
           title='My Coding Journey')


In [6]:
# fix navigation
def fixNav(fn ,
           navF = '../docs/View/html/nav.html',
           cssDir='../css/',
           jsDir='../js/',
           htmlDir='../html/'):
    with open(fn) as f:
        page = bs(f, 'html')
    with open(navF) as f:
        nav = bs(f, 'html')
    for a in nav.find_all('a'):
        a['href'] = htmlDir + a['href'].split('/')[-1]
    for h in page.body.find_all('script'):
        if not h['src'].startswith('http'):
            h['src'] = jsDir + h['src'].split('/')[-1]
    for h in page.head.find_all('link'):
        if not h['href'].startswith('http'):
            h['href'] = cssDir + h['href'].split('/')[-1]
    tag = page.find('script', id='navbar')
    try:
        tag.name = 'div'
        del tag['src']
    except:
        page.append(bs('<div id="navbar"></div>'))
    page.find('div', id='navbar').append(nav)
    with open(fn, "w", encoding='utf-8') as file:
        file.write(str(page))
    return page



In [7]:
files = !find ../docs/View/html -iname "*.html"

In [18]:
files

['../docs/View/html/index.html',
 '../docs/View/html/previois.html',
 '../docs/View/html/articles/windows0105.html',
 '../docs/View/html/articles/latex_html.html',
 '../docs/View/html/articles/add_user_to_sudo_list.html',
 '../docs/View/html/articles/filetypes.html',
 '../docs/View/html/articles/linuxtips.html',
 '../docs/View/html/articles/startup_chime.html',
 '../docs/View/html/articles/generate_vs_code_snippet.html',
 '../docs/View/html/articles/nav.html',
 '../docs/View/html/research_intersts.html',
 '../docs/View/html/general_intersts.html',
 '../docs/View/html/nav.html']

In [17]:
for f in files:
    fixNav(f)

AttributeError: 'NoneType' object has no attribute 'find_all'